In [ ]:
import os
import pickle as pkl
import csv
import math
import pandas as pd
import numpy as np

from kmapper import KeplerMapper, Cover
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

WDIRS = ['/Users/danielhasegan/workspace/netpyne-project/results/20210907-ES1500it/evaluation_0/',
    '/Users/danielhasegan/workspace/netpyne-project/results/20210907-ES1500it/evaluation_15/',
    '/Users/danielhasegan/workspace/netpyne-project/results/hpsearch-2021-09-06/best/1_run_168/evaluation_8/',]

NAMES = ['INIT', 'EVOL', 'STDP']


# wdir= '/Users/danielhasegan/workspace/netpyne-project/results/20210907-ES1500it/evaluation_0/'
# wdir= '/Users/danielhasegan/workspace/netpyne-project/results/20210907-ES1500it/evaluation_15/'
# wdir= '/Users/danielhasegan/workspace/netpyne-project/results/hpsearch-2021-09-06/best/1_run_168/evaluation_8/'

LENSES = []


resolution = 49.8 / 4
offset = 49.8 * 3
N = 10000 # total_length // 5

for wdir in WDIRS:
    print(wdir)

    sim_path = os.path.join(wdir, 'sim.pkl')

    with open(sim_path, 'rb') as f:
        sim = pkl.load(f)

    total_length = math.ceil((sim['simData']['spkt'][-1] - offset) / resolution)
    num_gids = int(max(sim['simData']['spkid']) + 1)
#     N = 10000 # total_length // 5
    M = num_gids

    X = np.zeros((N, M), dtype=np.short)

    for spkt, spkid in zip(sim['simData']['spkt'], sim['simData']['spkid']):
        if spkt < offset:
            continue

        t = math.floor((spkt - offset) / resolution)
        if t >= N:
            continue
        X[t][int(spkid)] = 1

    # Generate shape graph using KeplerMapper
    mapper = KeplerMapper(verbose=1)
    lens = mapper.fit_transform(X, projection=TSNE(2, perplexity=50, random_state=1))

    LENSES.append(lens)

In [ ]:
MOT_OUTS = []


for wdir in WDIRS:
    mot_path = os.path.join(wdir, 'MotorOutputs.txt')
    
    with open(mot_path) as f:
        mot_rews = [line for line in csv.reader(f, delimiter='\t')]

    y = np.zeros(N, dtype=np.short)

    for idx, row in enumerate(mot_rews):
        if float(row[0]) < offset:
            continue
        tstart = math.floor((float(row[0]) - offset) / resolution)

        if idx < len(mot_rews) - 1:
            tnext = math.floor((float(mot_rews[idx+1][0]) - offset) / resolution)
        else:
            tnext = N

        lmove, rmove = float(row[1]), float(row[2])
        move = 1 if lmove == rmove else (2 if lmove > rmove else 3)

        for t in range(tstart, tnext):
            if t >= N:
                continue
            y[t] = move
            
    MOT_OUTS.append(y)

In [ ]:
import matplotlib.pyplot as plt


for wdir, name, lens, y in zip(WDIRS, NAMES, LENSES, MOT_OUTS):
    print(name)

    S = 5
    plt.figure(figsize=(5,5))

    L = []

    for i in [2,3,1]:
        plt.scatter(lens[y == i][:,0], lens[y == i][:,1], 5, alpha=0.4)
        L.append(['stay', 'left', 'right'][i-1])

    plt.title('t-SNE on {} activations'.format(name))
    plt.tight_layout()

    plt.legend(L)
    plt.xticks([])
    plt.yticks([])
    plt.savefig('tSNE-{}.png'.format(name.lower()), dpi=300)

## Below is deprecated

In [ ]:
import matplotlib
matplotlib.rcParams['font.family']


In [ ]:
ACTS_EPS = []

# EPS_INTERVALS = [
#     [0, 20],
#     [20, 30],
#     [30, 40],
#     [40, 60],
#     [60, 100],
#     [100, 150],
#     [150, 200],
#     [200, 300],
#     [300, 501]]

EPS_INTERVALS = [
    [0, 50],
    [50, 100],
    [100, 200],
    [200, 501]]


for wdir in WDIRS:
    acts_path = os.path.join(wdir, 'ActionsPerEpisode.txt')
    
    with open(acts_path) as f:
        acts = [int(float(line[1])) for line in csv.reader(f, delimiter='\t')]

    y = np.zeros(N, dtype=np.short)
    
    current_eps = 0
    for idx, eps in enumerate(acts):
        interval_idx = [i for i,(s,e) in enumerate(EPS_INTERVALS) if s <= eps and eps < e][0]
        
        tstart = current_eps
        tend = tstart + eps  + (-3 if idx == 0 else 0)

        for t in range(tstart, tnext):
            if t >= N:
                continue
            y[t] = interval_idx
            
        current_eps = tend
            
    ACTS_EPS.append(y)

In [ ]:
for wdir, name, lens, y in zip(WDIRS, NAMES, LENSES, ACTS_EPS):
    print(name)

    plt.figure(figsize=(10,10))

    L = []

    for i,(s,e) in enumerate(EPS_INTERVALS):
        plt.scatter(lens[y == i][:,0], lens[y == i][:,1], 5, alpha=0.7)
        L.append('Acts per Eps in [{},{})'.format(s,e))

    plt.title('t-SNE on {} activations'.format(name))
    plt.tight_layout()

    plt.legend(L)
    plt.xticks([])
    plt.yticks([])
    plt.savefig('tSNE_actseps-{}.png'.format(name.lower()), dpi=300)

In [ ]:
import matplotlib.pyplot as plt

d = np.sum((lens[1:] - lens[:-1]) ** 2, axis=1)

plt.hist(d)

In [ ]:
graph = mapper.map(lens, X, cover=Cover(10, 0.8), clusterer=DBSCAN(eps=10.0))

In [ ]:
from dyneusr import DyNeuGraph

# Visualize the shape graph using DyNeuSR's DyNeuGraph                          
dG = DyNeuGraph(G=graph, y=y)
dG.visualize('dyneusr_stdp_p50.html')


In [ ]:
from dyneusr.tools import visualize_mapper_stages

fig, axes = visualize_mapper_stages(
    X, y=y, lens=lens, graph=graph, cover=mapper.cover,
    node_size=20, edge_size=0.5, edge_color='gray',
    layout="kamada_kawai",  figsize=(16, 4)
    )

plt.savefig('tSNE-stdp.png', dpi=300)
plt.show()

In [ ]:
lens[y == 1][:,0]